# Step 4: Model Development

---

Appropriate machine learning algorithms for classification problems like ours includes:

- Logistic Regression
- Decision Tree
- Random Forest
- k-Nearest Neighbors
- Support Vector Machine
- Naive Bayes

Considering the small number of samples in our dataset, the number of features, the binary nature of our problem, and the assignment constraints, we will develop and pick out the best algorithms among these 4:

- Logistic Regression
- Decision Tree
- Bagged Trees
- Random Forest

For individual model's performance evaluation, we will be using [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) to measure our models' performance and optimize them. While every sepsis case needs immediate attention, thus every potential sepsis patient needs close monitering, false positive predictions might take medical resources away from patients that might actually need it. In subsequent model development, if we see that recall and precision have different weights in our problem domain, we can switch to [F-beta](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fbeta_score.html#:~:text=The%20F-beta%20score%20is,recall%20in%20the%20combined%20score.) instead.

To get this performance, we will use K-Fold cross validation.

For regularization, we will be using Grid Search method to select and fine tune our hyperparameters.

To compare between the different model of different algorithms, we will use both $F_1$ score and [ROC-AUC](https://www.youtube.com/watch?v=4jRBRDbJemM). This will be done after we have developed and picked our best models, one for each of the above 4 algorithms. Skip to [Step 5. Model Selection](Step5.ModelSelection.ipynb) for more details.

---

**Table of Contents**

1. [Dataset](#data)
2. [Logistic Regression](#logistic)
3. [Decision Tree](#tree)
4. [Bagged Trees](#tree)
5. [Random Forest](#forest)

In [1]:
# Imports and environment setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')


from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, classification_report, confusion_matrix

We will store our developed models and their performance in the following variables:

In [2]:
clfs = dict()

## 1. Data Splitting<a id="data"></a>

For this section, we are using the processed datasets from [Step 3. Data Processing](Step3.DataProcessing.ipynb).

In [3]:
df_train = pd.read_csv("../data/processed_train.csv")
df_test = pd.read_csv("../data/processed_test.csv")

In [4]:
df_test.info()
df_test.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PRG     169 non-null    float64
 1   PL      169 non-null    float64
 2   PR      169 non-null    float64
 3   SK      169 non-null    float64
 4   TS      169 non-null    float64
 5   M11     169 non-null    float64
 6   BD2     169 non-null    float64
 7   Age     169 non-null    float64
dtypes: float64(8)
memory usage: 10.7 KB


,PRG,PL,PR,SK,TS,M11,BD2,Age
count,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02
mean,1.156209e-16,-3.310961e-16,-1.734313e-16,-8.146015e-17,-6.832142e-17,-1.072121e-15,5.255494e-17,-1.072121e-15
std,1.002972e+00,1.002972e+00,1.002972e+00,1.002972e+00,1.002972e+00,1.002972e+00,1.002972e+00,1.002972e+00
min,-1.208521e+00,-3.241433e+00,-4.708929e+00,-1.474053e+00,-9.760584e-01,-8.063583e+00,-1.980105e+00,-1.245210e+00
25%,-1.207359e+00,-6.956845e-01,2.720883e-02,-1.474053e+00,-9.760584e-01,-2.507388e-01,-7.312326e-01,-8.293165e-01
50%,6.858171e-02,-5.691907e-03,2.302438e-01,5.313517e-01,-9.760584e-01,1.268218e-01,-5.861043e-02,-3.491998e-01
75%,8.739056e-01,6.789913e-01,3.197079e-01,7.425575e-01,9.923200e-01,4.138384e-01,7.819177e-01,9.136708e-01
max,1.772338e+00,2.141800e+00,7.261358e-01,1.015063e+00,1.548606e+00,1.469355e+00,2.445566e+00,2.504717e+00


In [5]:
df_train.info()
df_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PRG     782 non-null    float64
 1   PL      782 non-null    float64
 2   PR      782 non-null    float64
 3   SK      782 non-null    float64
 4   TS      782 non-null    float64
 5   M11     782 non-null    float64
 6   BD2     782 non-null    float64
 7   Age     782 non-null    float64
 8   Sepsis  782 non-null    float64
dtypes: float64(9)
memory usage: 55.1 KB


,PRG,PL,PR,SK,TS,M11,BD2,Age,Sepsis
count,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.00000
mean,0.093303,0.120884,0.053327,0.066275,0.094252,0.079220,0.081804,0.107794,0.50000
std,0.981172,0.991983,1.024143,0.972240,0.993948,0.979384,1.007280,1.002119,0.50032
min,-1.739748,-3.875880,-3.277139,-3.720660,-1.784871,-2.680662,-2.664330,-1.288830,0.00000
25%,-0.756075,-0.554200,-0.471875,-0.524925,-0.325599,-0.588994,-0.642949,-0.713537,0.00000
50%,0.164328,0.120653,0.097826,-0.066467,-0.325599,0.143147,0.133515,-0.003753,0.50000
75%,0.803661,0.835510,0.787667,0.786756,0.522600,0.726554,0.911406,0.918774,1.00000
max,1.787925,2.066296,2.182884,2.050475,1.652611,2.195947,1.855361,2.256671,1.00000


In [6]:
# Separate our features (X) and target (y)
df_X = df_train.drop(["Sepsis"], axis=1)
df_y = df_train[["Sepsis"]]

# Splitting our training data into a train set and a validation set
train_X, val_X, train_y, val_y = train_test_split(
    df_X, df_y,
    shuffle=True,
    random_state=0,  # Ensure reproducible results
    test_size=0.2    # 80% Train - 20% Validation
)

print(f"train_X shape: {train_X.shape}")
print(f"train_y shape: {train_y.shape}")
print(f"val_X shape: {val_X.shape}")
print(f"val_y shape: {val_y.shape}")

train_X shape: (625, 8)
train_y shape: (625, 1)
val_X shape: (157, 8)
val_y shape: (157, 1)


Setting up K-Fold cross validation for our models:

In [7]:
cv = KFold(
    n_splits=4,     # Each fold is 20% of df_X
    shuffle=True,
    random_state=0  # Reproducible result
)

## 2. Logistic Regression<a id="logistic"></a>

We would develop and evaluate the following models to find the best Logistic Regression candidate for our problem:

- Logistic Regression with Linear Features (`clf_logistic["linear"]`);
- Logistic Regression with Polynomial Features of degree 2 (`clf_logistic["poly2"]`);
- Logistic Regression with Polynomial Features of degree 3 (`clf_logistic["poly3"]`);
- Logistic Regression with Polynomial Features of degree 4 (`clf_logistic["poly4"]`).

We are experimenting with both $L_1$ (lasso) and $L_2$ (ridge) regularization to find out which method is more appropriate. And although findings in [Step 1. EDA](Step1.EDA.ipynb) suggest that we stop at Polynomial Features of degree 3, we are still experimenting with degree 4 because we introduced regularization into our Logistic Regression model. However, we will not go any higher than degree 4, since it is too computationally expensive.

In [8]:
clf = LogisticRegression(
    solver="liblinear",       # Good for our small dataset
    class_weight="balanced",
    max_iter=1_000,
    random_state=0            # Reproducible result
)

clfs_logistic = dict()

### 2.1. Hyperparameters

- Regularization strength $\lambda$ (lambda)
- Regularization type: $L_1$ or $L_2$

In [9]:
params = {
    "C": 1 / np.logspace(-5, 2, num=100),
    "penalty": ["l1", "l2"]
}

### 2.2. Logistic Regression with Linear Features

In [25]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X, train_y.values.ravel())

pred_y = clfs_logistic["linear"].predict(val_X)
clfs_logistic["linear"] = grid_clf

In [26]:
print(grid_clf.best_score_)
print(classification_report(val_y, pred_y))

0.8166438146598718
              precision    recall  f1-score   support

         0.0       0.93      0.79      0.86        82
         1.0       0.80      0.93      0.86        75

    accuracy                           0.86       157
   macro avg       0.87      0.86      0.86       157
weighted avg       0.87      0.86      0.86       157



In [ ]:
fig, axis = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.heatmap(confusion_matrix(val_y, pred_y), annot=True, cmap='Blues', fmt='g', ax=axis)
axis.set_xlabel('Predicted Values')
axis.set_ylabel('True Values')
axis.set_title('Confusion Matrix for Logistic Regression Model')

axis.set_xticks([0.5, 1.5])
axis.set_xticklabels(['Negative', 'Positive'])
axis.set_yticks([0.5, 1.5])
axis.set_yticklabels(['Negative', 'Positive'])

plt.show()
fig.savefig("../images/LogisticRegression.png")
print(classification_report(val_y, pred_y))

### 2.3. Logistic Regression with Degree 2 Polynomial Features

In [11]:
# Degree 2 Polynomial Features:
poly = PolynomialFeatures(2)
train_X_poly2 = poly.fit_transform(train_X)
val_X_poly2 = poly.transform(val_X)

# We need to scale the data again after transforming it to polynomial features
scaler = StandardScaler()
train_X_poly2 = scaler.fit_transform(train_X_poly2)
val_X_poly2 = scaler.transform(val_X_poly2)

In [12]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X_poly2, train_y.values.ravel())

clfs_logistic["poly2"] = grid_clf

### 2.4. Logistic Regression with Degree 3 Polynomial Features

In [13]:
# Degree 3 Polynomial Features:
poly = PolynomialFeatures(3)
train_X_poly3 = poly.fit_transform(train_X)
val_X_poly3 = poly.transform(val_X)

# We need to scale the data again after transforming it to polynomial features
scaler = StandardScaler()
train_X_poly3 = scaler.fit_transform(train_X_poly3)
val_X_poly3 = scaler.transform(val_X_poly3)

In [14]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X_poly3, train_y.values.ravel())

clfs_logistic["poly3"] = grid_clf

### 2.5. Logistic Regression with Degree 4 Polynomial Features

In [15]:
# Degree 4 Polynomial Features:
poly = PolynomialFeatures(4)
train_X_poly4 = poly.fit_transform(train_X)
val_X_poly4 = poly.transform(val_X)

# We need to scale the data again after transforming it to polynomial features
scaler = StandardScaler()
train_X_poly4 = scaler.fit_transform(train_X_poly4)
val_X_poly4 = scaler.transform(val_X_poly4)

In [16]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X_poly4, train_y.values.ravel())

clfs_logistic["poly4"] = grid_clf

### 2.6. Observation

All models seem to have good fit on the problem, however:

- $L_1$ (Lasso) Regularization clearly is not a good choice for our problem, as the larger $\lambda$ (lambda) grows, the worse our models' performance gets, regardless of which degree of polynomial feature it is. This is likely because we have only a few features, and all of them are significant, while Lasso Regularization tends to feature select.
- Thus $L_2$ (Ridge) Regularization is more appropriate for our problem.
- The best performing model is `clf_logistic4_l2`, in contrary to our hypothesis in [Step 1. EDA](Step1.EDA.ipynb), but this is because we have applied regularization.

### 2.8. Best Logistic Regression Model

## 3. Decision Tree<a id="tree"></a>

### 3.1. Hyperparameters

In [17]:
params = {
    "max_depth": np.arange(2, 300, 50),
    "min_samples_split": np.arange(2, 50, 5),
}

### 3.2. Model

In [18]:
clf = DecisionTreeClassifier(
    criterion="gini",         # Good for balanced data (we have already up-sampled out data so it is now balanced)
    class_weight="balanced",
    random_state=0            # Reproducible result
)

grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X, train_y.values.ravel())

GridSearchCV(cv=KFold(n_splits=4, random_state=0, shuffle=True),
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=0),
             param_grid={'max_depth': array([  2,  52, 102, 152, 202, 252]),
                         'min_samples_split': array([ 2,  7, 12, 17, 22, 27, 32, 37, 42, 47])},
             scoring='f1')